In [128]:
import sys
sys.path.append("/home/chenlidong/polyAttn")
import numpy as np
import pandas as pd
from rdkit import Chem
from utils import molecule2graph

In [3]:
element_names = ['*', 'C', 'N', 'O', 'S', 'F', 'Si', 'P', 'Cl', 'Br','I']

In [130]:
homopolymer_data = pd.read_csv("/home/chenlidong/polyAttn/data/homopolymer/homopolymer.csv")
copolymer_data = pd.read_csv("/home/chenlidong/polyAttn/data/copolymer/copolymer.csv")

In [5]:
def get_info_from_copolymer_format(copolymer_str):
    info = copolymer_str.split("|")
    return info

In [16]:
homopolymer_data[:370]

,SMILES,Ea,Ei,Tg
0,[*]CC([*])C,0.4343,6.1850,NaN
1,[*]CC([*])F,0.8740,7.6332,NaN
2,[*]CC([*])(F)F,1.1415,8.1531,NaN
3,[*]C(F)C([*])(F)F,1.5240,8.5986,NaN
4,[*]CCC(F)(F)C([*])(F)F,0.4489,9.0178,NaN
...,...,...,...,...
365,[*]CCC(=O)OC(=O)CC(C=C1)=CC=C1C(C=C2)=CC=C2[*],1.5387,6.1951,NaN
366,[*]C(=O)NONC(=O)NC(S1)=CC=C1N[*],1.3385,5.6948,NaN
367,[*]C(=S)C(=O)CCNC(C=C1)=CC=C1C(=S)C(C=C2)=CC=C...,3.8901,5.8838,NaN
368,[*]C(F)(F)C(F)(F)C(S1)=CC=C1C(S2)=CC=C2[*],2.7568,5.8176,NaN


# 下面的代码用于统计两个数据集里的元素组成

In [19]:
df = homopolymer_data[370:]
# df = copolymer_data
additional_set = set()
num = [0] * len(element_names)
first_column = df.iloc[:, 0]
for index, row in first_column.iteritems():
    mol = Chem.MolFromSmiles(get_info_from_copolymer_format(row)[0])
    atom_elements = set([atom.GetSymbol() for atom in mol.GetAtoms()])
    for element in atom_elements:
        if element in element_names:
            index = element_names.index(element)
        else:
            index = len(element_names) - 1
            additional_set.add(element)
        num[index] += 1
print(additional_set)
print(num)

set()
[96, 96, 55, 90, 10, 13, 4, 4, 4, 1, 0]


/tmp/ipykernel_403518/1524551970.py:7: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, row in first_column.iteritems():


# 下面的代码是统计数据集里两个目标的平均数和误差

# 下面的代码是polygnn里的数据增强

In [25]:
def n_to_subtract(atoms_removed_inds, atom_ind):
    return int(np.argwhere(np.sort(atoms_removed_inds + [atom_ind]) == atom_ind)[0][0])

In [26]:
def bind_frags(
    m1,
    m1_tails,
    m2,
    m2_heads,
    m1_connectors,
    m2_connectors,
    bond_types,
):
    """
    Bind *mol* objects, m1 and m2, together with a bond of type 'bond_type' at connection points m1_tail and m2_head
    """

    combo_mol = Chem.rdmolops.CombineMols(m1, m2)
    em = Chem.EditableMol(combo_mol)
    removed_atoms = [np.inf]  # init with 'inf' so that n_to_subtract will be 0 to start
    for (m1_connector, m2_connector, m1_tail, m2_head, bond_type) in zip(
        m1_connectors, m2_connectors, m1_tails, m2_heads, bond_types
    ):
        em.AddBond(
            m1_connector - n_to_subtract(removed_atoms, m1_connector),
            m2_connector
            + m1.GetNumAtoms()
            - n_to_subtract(removed_atoms, m2_connector + m1.GetNumAtoms()),
            bond_type,
        )
        em.RemoveAtom(m1_tail - n_to_subtract(removed_atoms, m1_tail))
        removed_atoms.append(m1_tail)
        em.RemoveAtom(
            m2_head
            + m1.GetNumAtoms()
            - n_to_subtract(removed_atoms, m2_head + m1.GetNumAtoms())
        )
        removed_atoms.append(m2_head + m1.GetNumAtoms())

    new_mol = em.GetMol()
    try:
        Chem.SanitizeMol(new_mol)
        return new_mol
    except:
        print(f"!!! Binding failed for {Chem.MolToSmiles(combo_mol)} !!!")
        return None

In [27]:
def get_star_inds(mol):
    if type(mol) == str:
        mol = Chem.MolFromSmiles(mol)
    inds = tuple(mol.GetSubstructMatches(Chem.MolFromSmarts("[#0]~*")))
    inds = tuple(zip(*inds))
    if len(inds[0]) != 2:
        raise ValueError(
            "Invalid repeat unit. It is likely that at least one star contains more than one bond."
        )
    return list(inds[0]), list(inds[1])

In [33]:
class LinearPol(Chem.rdchem.Mol):
    """
    Linear Polymer class.

    Attributes:
        mol (Chem.rdchem.Mol): The RDKit molecule of the parent polymer.
        star_inds (list): List of star atom indices in the parent polymer. Sorted from smallest to largest index.
        connector_inds (list): List of connector atom indices in the parent polymer. Sorted from smallest to largest index.
        periodic_bond_type (Chem.rdchem.BondType): The bond type of the periodic bond in the parent polymer.
    """

    def __init__(self, mol):
        """
        Initializes the LinearPol object.

        Args:
            mol (str or Chem.rdchem.Mol): SMILES string or RDKit molecule of the parent polymer.
        """
        if isinstance(mol, str):
            self.SMILES = mol
            self.mol = Chem.MolFromSmiles(mol)
        else:
            self.mol = mol

        self.star_inds, self.connector_inds = get_star_inds(self.mol)
        self.periodic_bond_type = self.get_pbond_type()

    def get_pbond_type(self):
        """
        Returns the bond type of the periodic bond.

        Returns:
            Chem.rdchem.BondType: The bond type of the periodic bond.

        Raises:
            ValueError: If the bond types of the periodic bond are mismatching.
        """
        bond1_type = self.mol.GetBondBetweenAtoms(self.star_inds[0], self.connector_inds[0]).GetBondType()
        bond2_type = self.mol.GetBondBetweenAtoms(self.star_inds[1], self.connector_inds[1]).GetBondType()
        if bond1_type != bond2_type:
            raise ValueError("Invalid repeat unit. Periodic bond types are mismatching.")
        return bond1_type

    def multiply(self, n):
        """
        Return a LinearPol which is n times repeated from itself
        """
        add_lp = LinearPol(self.mol)
        for _ in range(n - 1):
            add_lp = LinearPol(
                bind_frags(
                    self.mol,
                    [self.star_inds[1]],
                    add_lp.mol,
                    [add_lp.star_inds[0]],
                    [self.connector_inds[1]],
                    [add_lp.connector_inds[0]],
                    [self.periodic_bond_type],
                )
            )
        return add_lp

In [37]:
lp = LinearPol("[*]Oc1ccc(-c2ccc(C([*])=O)cc2)cc1")
pm = lp.multiply(3).mol
Chem.MolToSmiles(pm)

'*Oc1ccc(-c2ccc(C(=O)Oc3ccc(-c4ccc(C(=O)Oc5ccc(-c6ccc(C(*)=O)cc6)cc5)cc4)cc3)cc2)cc1'

# 下面的代码是测试概率边

In [142]:
smiles_list = copolymer_data.iloc[0:5,:].SMILES.to_list()

In [145]:
graph = molecule2graph.mol2graph(smiles_list)

In [149]:
graph.get_components()[-2]

[(1, 42), (43, 54), (97, 54), (151, 42), (193, 54)]